In [1]:
%matplotlib inline
import re
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from joblib import Parallel, delayed
from tqdm.notebook import tqdm as tqdm
from tqdm.notebook import trange
import contextlib
import joblib
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import edward2 as ed
import tensorflow as tf
from scipy.special import digamma
from pickle import dump, load
from scipy.sparse import csr_matrix
import tensorflow_probability as tfp
import os
import sys
import time

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
# tensorflow does not work with new numpy versions
assert np.__version__  < '1.20'



In [2]:
tqdm.pandas()

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/senkin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [55]:
# data_proc = pd.read_pickle('data_proc.pkl')
# data_enc = pd.read_pickle('data.pkl')
# with open('word_to_idx.pkl', 'rb') as f:
#     words_to_idx = load(f)

In [41]:
seed = 42
data = pd.read_csv('nips-papers/papers.csv')
data = data[['paper_text']]
data = data.sample(n=10, random_state=42)

In [42]:
# data.head()

In [43]:
def func(text):
    text = text.lower()
    text = re.sub(r'(\d+)', '', text)
    text = re.sub(r'(\n)|(\t)', ' ', text)
    text = text.translate({ ord(c): None for c in string.punctuation })
    text = text.strip()
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    text = [i for i in tokens if not i in stop_words and len(i) > 1]
    text = [lemmatizer.lemmatize(word) for word in text]
    return text

In [44]:
data_proc = data['paper_text'].progress_apply(func)

  0%|          | 0/10 [00:00<?, ?it/s]

In [45]:
data_proc.head()

509     [independent, component, analysis, identificat...
2576    [nearmaximum, entropy, model, binary, neural, ...
6362    [nearestneighbor, sample, compression, efficie...
6173    [efficient, highorder, interactionaware, featu...
6552    [multioutput, polynomial, network, factorizati...
Name: paper_text, dtype: object

In [46]:
words_to_idx = {}
idx_to_word = {}
def encode(text, words_to_idx, idx_to_word):
    text_enc = np.empty(len(text), dtype=int)
    for i, word in enumerate(text):
        if word not in words_to_idx:
            idx = words_to_idx[word] = len(words_to_idx)
            idx_to_word[idx] = word
        else:
            idx = words_to_idx[word]
        text_enc[i] = idx
    return text_enc
data_enc = data_proc.progress_apply(encode, args=(words_to_idx, idx_to_word))

  0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
# data_proc.to_pickle('data_proc.pkl')
# data_enc.to_pickle('data.pkl')

# with open('word_to_idx.pkl', 'wb') as f:
#     dump(words_to_idx, f)

In [56]:
K = 5
D = len(data_enc)
Ns = data_enc.apply(lambda x: len(x)).to_numpy().astype(np.int)
N = Ns.sum()
V = len(words_to_idx)

In [57]:
def create_sparse(data, alpha, D, K, V):
    Ns = np.empty(D, dtype=np.int)
     
    for i, doc in enumerate(data):
        Ns[i] = len(doc)
    
    N = Ns.sum()
    rows = np.empty(N, dtype=np.int64)
    cols = np.empty(N, dtype=np.int64)
    v_cols = np.empty(N, dtype=np.int64)
    
    last_idx = 0
    
    for i, doc in tqdm(enumerate(data), total=D):
        n = len(doc)
        rows[last_idx:last_idx+n] = i
        cols[last_idx:last_idx+n] = np.arange(n, dtype=np.int64)
        v_cols[last_idx:last_idx+n] = doc
        last_idx += n
        
    N_max = Ns.max()
    phi_rows = np.repeat(rows, K)
    phi_cols = np.repeat(cols, K)
    phi3 = np.tile(np.arange(K), len(rows))  
    phi_indices = np.vstack((phi_rows, phi_cols, phi3)).T
    phi = tf.sparse.SparseTensor(phi_indices, values=tf.fill((len(phi_indices),), 1/K), dense_shape=(D, N_max, K))
    phi = tf.sparse.reorder(phi)
    gamma_v_cols = np.repeat(v_cols, K)
    gamma = tf.expand_dims(alpha, 0) + (Ns / K).reshape(-1, 1)

    beta_indices = np.vstack((phi3, gamma_v_cols)).T
    gamma_indices = np.vstack((phi_rows, phi3)).T
    w_indices = np.vstack((phi_rows, phi_cols, phi3, gamma_v_cols)).T
    tmp = tf.sparse.SparseTensor(w_indices, phi.values, dense_shape=(D, N_max, K, V))
    tmp = tf.sparse.reorder(tmp)
    w_indices = tmp.indices
    lmbd = tf.fill((K, V), 1/V)
    return phi, gamma, lmbd, tf.constant(beta_indices, dtype=tf.int64), tf.constant(gamma_indices, dtype=tf.int64), tf.constant(w_indices, dtype=tf.int64), N_max

In [58]:
# alpha_n = np.random.rand(K).astype(np.float32)
# eta_n = np.random.rand(1).astype(np.float32)
# beta_n = np.random.dirichlet(np.random.rand(V).astype(np.float32), size=K).astype(np.float32)
# phi_n = [np.full((n, K), 1/K).astype(np.float32) for n in Ns]
# gamma_n = alpha_n.reshape(1, -1) + Ns.reshape(-1, 1) / K
# gammma_n = gamma_n.astype(np.float32)
# lmbd_n = np.full((K, V), eta_n)
# lmbd_n = lmbd_n.astype(np.float32)

In [ ]:
# It tries to estimate also smoothed params, but eta tends to go negative, breaking all the stuff.
# Maybe loss is incorrect

class Positive(tf.keras.constraints.Constraint):
    def __call__(self, w):
        return w * tf.cast(tf.math.greater(w, 0.), w.dtype)   
    
@tf.function
def calc_elbo(alpha, beta, eta, phi, gamma, lmbd, w_indices, D, K, N_max, V):
    digamma = tf.math.digamma(gamma) - tf.math.digamma(tf.math.reduce_sum(gamma, axis=1, keepdims=True))
    ta = tf.math.lgamma(tf.math.reduce_sum(alpha)) - \
                                          tf.math.reduce_sum(tf.math.lgamma(alpha)) + \
                                          tf.math.reduce_sum(tf.expand_dims(alpha - 1, 0)*digamma, axis=1)
    zt = tf.sparse.reduce_sum(tf.expand_dims(digamma, 1)*phi, axis=(1, 2))
    wzb = tf.math.reduce_sum(tf.expand_dims(tf.math.log1p(beta), axis=0) * tf.sparse.reduce_sum(tf.SparseTensor(w_indices, phi.values, dense_shape=(D, N_max, K, V)), axis=(1)), axis=(1, 2))
    bl = tf.math.reduce_sum((eta-1)*tf.math.reduce_sum(tf.math.digamma(lmbd) - tf.math.digamma(tf.math.reduce_sum(lmbd, axis=1, keepdims=True)), axis=1) + \
         tf.math.lgamma(eta*V) - V*tf.math.lgamma(eta))
    qt = -tf.math.lgamma(tf.math.reduce_sum(gamma, axis=1)) + tf.math.reduce_sum(tf.math.lgamma(gamma), axis=1) - tf.math.reduce_sum((gamma-1)*digamma, axis=1)
    phi_log_phi = tf.sparse.SparseTensor(phi.indices, phi.values*tf.math.log1p(phi.values), dense_shape=[D, N_max, K])
    qz = tf.sparse.reduce_sum(phi_log_phi, axis=(1, 2))
    elbo = tf.math.reduce_sum(ta + zt + qt + qz)
    return elbo

@tf.function
def e_step_it(alpha, beta_mod, eta, phi, gamma, lmbd, gamma_indices, w_indices, D, K, N_max, V):
    # phi
    dg = tf.math.exp(tf.math.digamma(gamma) - tf.math.digamma(tf.math.reduce_sum(gamma, axis=1, keepdims=True)))
    phi = tf.sparse.SparseTensor(phi.indices, beta_mod*tf.gather_nd(dg, gamma_indices), dense_shape=[D, N_max, K])
    phi /= tf.sparse.reduce_sum(phi, axis=2, keepdims=True) + 1e-5
    # gamma
    gamma = tf.expand_dims(alpha, 0) + tf.sparse.reduce_sum(phi, axis=1)
    # lambda
    lmbd = eta + tf.sparse.reduce_sum(tf.SparseTensor(w_indices, phi.values, dense_shape=(D, N_max, K, V)), axis=(0, 1))
    
    gamma.set_shape((D, K))
    lmbd.set_shape((K, V))
    return phi, gamma, lmbd

@tf.function
def e_step(alpha, beta, eta, phi, gamma, lmbd, beta_indices, gamma_indices, w_indices, D, K, N_max, V, max_it=1000, rtol=1e-03, atol=1e-03):      
    beta_mod = tf.gather_nd(beta, beta_indices)

    np_isclose = lambda elbo_old, elbo: np.allclose(elbo_old, elbo, rtol=rtol, atol=atol)
    tf_cond = lambda i, elbo_old, elbo, args: tf.logical_and(i < max_it, tf.logical_or(i == 0, tf.logical_not(tf.numpy_function(np_isclose, [elbo_old, elbo], tf.bool))))
    
    @tf.function
    def tf_body(i, elbo_old, elbo, args):
        phi, gamma, lmbd = args
        i = i + 1
        elbo_old = tf.identity(elbo)
        phi, gamma, lmbd = e_step_it(alpha, beta_mod, eta, phi, gamma, lmbd, gamma_indices, w_indices, D, K, N_max, V)
        elbo = calc_elbo(alpha, beta, eta, phi, gamma, lmbd, w_indices, D, K, N_max, V)
        args = (phi, gamma, lmbd)
        return (i, elbo_old, elbo, args)
    
    i = tf.constant(0, name='e_loop_counter')
    elbo = tf.constant(np.inf, dtype=tf.float32)
    elbo_old = tf.identity(elbo)
    args = (phi, gamma, lmbd)
    
    i, _, elbo, args = tf.while_loop(tf_cond, tf_body, [i, elbo_old, elbo, args], parallel_iterations=1)
    phi, gamma, lmbd = args
    return phi, gamma, lmbd

@tf.function
def m_step(alpha, alpha_2, beta, eta, phi, gamma, lmbd, w_indices, D, K, N_max, V, opt, max_it=1000, rtol=1e-03, atol=1e-03):
    # beta
    beta = tf.sparse.reduce_sum(tf.SparseTensor(w_indices, phi.values, dense_shape=(D, N_max, K, V)), axis=(0, 1))
    beta /= tf.math.reduce_sum(beta, axis=1, keepdims=True) + 1e-5
    # alpha
    digamma = tf.math.reduce_sum(tf.math.digamma(gamma) - tf.math.digamma(tf.math.reduce_sum(gamma, axis=1, keepdims=True)), axis=0)
    
    @tf.function
    def alpha_elbo(alpha, digamma):
        val = tf.math.reduce_sum(tf.math.lgamma(tf.math.reduce_sum(alpha)) - \
                                          tf.math.reduce_sum(tf.math.lgamma(alpha)) + \
                                          tf.math.reduce_sum(tf.expand_dims(alpha - 1, 0)*digamma, axis=1))
        return val
    
    alpha_it_cond = lambda i, digamma: i < K
    
    np_isclose = lambda elbo_old, elbo: np.allclose(elbo_old, elbo, rtol=rtol, atol=atol)
    alpha_cond = lambda i, elbo_old, elbo, digamma: tf.logical_and(i < max_it, tf.logical_or(i == 0, tf.logical_not(tf.numpy_function(np_isclose, [elbo_old, elbo], tf.bool))))
    
    @tf.function
    def alpha_it_body(i, digamma):        
        def loss_func():
            ta = tf.TensorArray(alpha.dtype, size=K, dynamic_size=False)
            alpha_copy = ta.unstack(alpha_2)
            alpha_copy = alpha_copy.write(i, tf.math.maximum(alpha[i], 1e-5))
            alpha_copy = alpha_copy.stack()
            alpha.assign(alpha_copy)
            alpha_2.assign(alpha_copy)
            a = tf.math.reduce_sum((alpha - 1)*digamma)
            res = -(D*(tf.math.lgamma(tf.math.reduce_sum(alpha)) - tf.math.reduce_sum(tf.math.lgamma(alpha))) + a)
            return res
        losses = tfp.math.minimize(loss_func, max_it, opt, convergence_criterion=tfp.optimizer.convergence_criteria.LossNotDecreasing(rtol=rtol, atol=atol), 
                                   trainable_variables=[alpha])
        
        ta = tf.TensorArray(alpha.dtype, size=K, dynamic_size=False)
        alpha_copy = ta.unstack(alpha_2)
        alpha_copy = alpha_copy.write(i, tf.math.maximum(alpha[i], 1e-5))
        alpha_copy = alpha_copy.stack()
        alpha.assign(alpha_copy)
        
        return i + 1, digamma
    
    @tf.function
    def alpha_body(i, elbo_old, elbo, digamma):
        i = i + 1
        j = tf.constant(0)
        elbo_old = tf.identity(elbo)
        j, _ = tf.while_loop(alpha_it_cond, alpha_it_body, [j, digamma], parallel_iterations=1)
#         tf.print(alpha, output_stream=sys.stdout)
        elbo = alpha_elbo(alpha, digamma)
        return (i, elbo_old, elbo, digamma)
    
    i = tf.constant(0)
    elbo = tf.constant(np.inf, dtype=tf.float32)
    elbo_old = tf.identity(elbo)
    i, _, elbo, _ = tf.while_loop(alpha_cond, alpha_body, [i, elbo_old, elbo, digamma], parallel_iterations=1)
    
    di_lmbd = tf.math.reduce_sum(tf.math.digamma(lmbd) - tf.math.digamma(tf.math.reduce_sum(lmbd, axis=1, keepdims=True))) 
    # eta
    def loss_func():
        loss_v = -((eta-1)*di_lmbd + K*(tf.math.lgamma(eta*V) - V*tf.math.lgamma(eta)))
        return loss_v
    
    losses = tfp.math.minimize(loss_func, max_it, opt, convergence_criterion=tfp.optimizer.convergence_criteria.LossNotDecreasing(rtol=rtol, atol=atol), trainable_variables=[eta])
    eta.assign(tf.math.maximum(eta, 0.1))
    return beta

def train(data, D, K, V, max_it=1000, seed=42, rtol=1e-3, atol=1e-3):
    start = time.time()
    alpha = tf.random.uniform((K,))
    eta = tf.random.uniform((1,))
    beta = tf.convert_to_tensor(np.random.dirichlet(np.random.rand(V), size=K), dtype=tf.float32)
    opt = tf.optimizers.Adam(1e-3)
    
    phi, gamma, lmbd, beta_indices, gamma_indices, w_indices, N_max = create_sparse(data, alpha, D, K, V)
    N_max = N_max.item()
    
    alpha = tf.Variable(alpha, constraint=Positive())
    alpha_2 = tf.Variable(tf.identity(alpha), trainable=False)
    eta = tf.Variable(eta, constraint=Positive())

    np_isclose = lambda elbo_old, elbo: np.allclose(elbo_old, elbo, rtol=1e-3, atol=1e-3)
    train_cond = lambda i, elbo_old, elbo, args: tf.logical_and(i < max_it, tf.logical_or(i == 0, tf.logical_not(tf.numpy_function(np_isclose, [elbo_old, elbo], tf.bool))))
    
    @tf.function
    def train_body(i, elbo_old, elbo, args):
        i = i + 1
        beta, phi, gamma, lmbd = args
        elbo_old = elbo
        tf.print(i, ': E step', output_stream=sys.stdout, end='\t')
        phi, gamma, lmbd = e_step(alpha, beta, eta, phi, gamma, lmbd, beta_indices, gamma_indices, w_indices, D, K, N_max, V, atol=atol, rtol=rtol)
        tf.print('M step', output_stream=sys.stdout, end='\t')
        beta = m_step(alpha, alpha_2, beta, eta, phi, gamma, lmbd, w_indices, D, K, N_max, V, opt, atol=atol, rtol=rtol)
        elbo = calc_elbo(alpha, beta, eta, phi, gamma, lmbd, w_indices, D, K, N_max, V)
        tf.print('ELBO =', elbo, output_stream=sys.stdout)
        args = (beta, phi, gamma, lmbd)
        return i, elbo_old, elbo, args
    
    i = tf.constant(0)
    args = (beta, phi, gamma, lmbd)
    elbo = tf.constant(np.inf, dtype=tf.float32)
    elbo_old = tf.identity(elbo)
    print('Preparing...')
    i, elbo_old, elbo, args = tf.while_loop(train_cond, train_body, [i, elbo_old, elbo, args], parallel_iterations=1)
    beta, phi, gamma, lmbd = args
    tf.print('Converged in', i,  'iterations', output_stream=sys.stdout)
    end = time.time()
    print('Time:', end-start, 's')
    return alpha, beta, eta, phi, gamma, lmbd, elbo

alpha, beta, eta, phi, gamma, lmbd, elbo = train(data_enc, D, K, V)

  0%|          | 0/4000 [00:00<?, ?it/s]

Preparing...
1 : E step	M step	

In [88]:
# Get topic top words
for h in range(K):
    print(([idx_to_word[i] for i in tf.math.top_k(beta[h], k=10)[1].numpy()]))

<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.4967644], dtype=float32)>

In [28]:
# Generate doc
theta_d = ed.Dirichlet(alpha)
for i in range(100):
    z_dn = ed.Categorical(probs=theta_d)
    phi_z = 
    w_dn = ed.Categorical(probs=beta[z_dn])
    print(idx_to_word[w_dn.numpy()])

processing
hmm
extreme
activation
full
dead
equilibrium
case
weight
qn
specific
exist
symmetric
maxproduct
plain
sampled
like
fii
gatsby
assigns
problem
leaf
weightsharing
distribution
respectively
hinton
algorithm
similar
unobserved
using
sequence
standard
bk
function
network
moemicrosoft
backpropagation
regression
function
operation
rissanen
one
set
science
end
music
university
mean
sequence
example
node
stationary
rtdp
corresponds
weight
process
ensure
yield
representation
update

wi
hidden
linear
section
distribution
ie
fpf
wn
take
paper
sign
marcus
choice
sli
vague
chosen
covu
across
method
pair
sn
taking
one
perturbing
mixture
examined
study
like
structure
structured
rest
stochastic
intelligence
ancestor
total
gibbsmh
consider
phase
split
regularization
depend
mechanic
logmse
bound
larger
ann
multistate
tensor
us
reinforcement
weight
operator
vector
data
prerequisite
interference
state
transitioned
suboptimal
qjz
organisation
remainder
rate
neural
appear
finite
parameter
fokker

In [61]:
# MAX_IT = 10
# EPS = 0.001
# tf.executing_eagerly()
# optim = tf.keras.optimizers.Adam(1e-3)
# # B = ed.Dirichlet(concentration=tf.fill([K, V], 0.1), name="topics")
# # Z = ed.DirichletMultinomial(tf.convert_to_tensor(Ns), concentration=tf.fill([D, K], 0.1))
# alpha = np.copy(alpha_n).astype(np.float32)
# eta = np.copy(eta_n).astype(np.float32)

# beta = np.copy(beta_n).astype(np.float32)
# phi = [np.full((n, K), 1/K).astype(np.float32) for n in Ns]
# gamma = np.copy(gamma_n).astype(np.float32)
# lmbd = np.copy(lmbd_n).astype(np.float32)

# bb = None
# gg = None
# ww = None

# class Positive(tf.keras.constraints.Constraint):
#     def __call__(self, w):
#         return w * tf.cast(tf.math.greater(w, 0.), w.dtype)

# bb = []
# for it in trange(MAX_IT):
#     bb2 = []
#     gg2 = []
#     ww2 = []
#     print('before', gamma)
#     for d in range(D):
#         for n in range(Ns[d]):
#             for i in range(K):
#                 phi[d][n, i] = beta[i, data_enc.iloc[d][n]] * np.exp(digamma(gamma[d, i]) - digamma(np.sum(gamma[d])))
#         phi[d] /= np.sum(phi[d], axis=-1, keepdims=True) + 1e-5
        
#         for i in range(K):
#             gamma[d, i] = alpha[i] + np.sum(phi[d][:, i])
#     print('after', gamma)

#     lmbd = np.full((K, V), eta)
#     for i in range(K):
#         for j in range(V):
#             for d in range(D):
#                 mask = (data_enc.iloc[d] == j)
#                 lmbd[i, j] += np.sum(phi[d][:, i]*mask)
    
     
# #     if bb is None:
# #         bb = bb2
# #         gg = gg2
# #         ww = ww2
# #         break
                
#     alpha_t = tf.Variable(alpha, trainable=True, constraint=Positive())
#     gamma_t = tf.convert_to_tensor(gamma, dtype=tf.float32)
    
    
#     def f_x():
#         g_term = tf.math.reduce_sum(tf.expand_dims((alpha_t - 1), 0)*(tf.math.digamma(gamma_t) - 
#                                                    tf.math.digamma(tf.math.reduce_sum(gamma_t, axis=1, keepdims=True))), axis=1)
#         loss = -tf.math.reduce_sum(tf.math.lgamma(tf.math.reduce_sum(alpha_t)) - tf.math.reduce_sum(tf.math.lgamma(alpha_t)) + g_term)
#         return loss
    
#     for itt in range(10):
#         for i in range(K):
#             for itt1 in range(50):
#                 #with tf.GradientTape() as tape:
#                 optim.minimize(f_x, [alpha_t])
# #                 grads = tape.gradient(loss, opt_a)
# #                 optim.apply_gradients([(grads, opt_a)])
#                 alpha[i] = alpha_t.numpy()[i]
#                 np.nan_to_num(alpha, copy=False, nan=1e-5)
#                 alpha_t.assign(alpha)
#         print(alpha_t)
#     beta = (lmbd - eta) / (np.sum(lmbd - eta, axis=-1, keepdims=True) + 1e-5)
#     break
    
#     eta_t = tf.Variable(eta, trainable=True, constraint=Positive())
    
#     @tf.function
#     def f_eta():
#         loss = K*((eta_t-1)*(tf.math.digamma(eta_t) - tf.math.digamma(eta_t*V)) + tf.math.lgamma(eta_t*V) - V*tf.math.lgamma(eta))
#         return loss
    
#     for itt1 in range(50):
#         optim.minimize(f_eta, [eta_t])
#     eta = eta_t.numpy()

    

  0%|          | 0/10 [00:00<?, ?it/s]

before [[1948.2192  1948.2751 ]
 [1251.2192  1251.2751 ]
 [ 796.21924  796.27515]]
after [[1558.9817 1524.9785]
 [1031.4255 1035.2314]
 [ 641.7817  640.5525]]
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0.7681685 , 0.82452303], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0.8167327 , 0.87247795], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0.8638188, 0.9190715], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0.90961564, 0.96447587], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0.95430493, 1.0088501 ], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0.99804085, 1.0523303 ], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1.0409529, 1.0950313], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1.0831494, 1.137053 ], dtype=float32)>